In [1]:
import numpy as np
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.gleu_score import sentence_gleu
from nltk.translate.meteor_score import meteor_score
from google.transliteration import transliterate_text
import re

In [2]:
file_path_1 = "../datasets/HINGE dataset/test.csv"
file_path_2 = "../datasets/HINGE dataset/valid.csv"
df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

In [3]:
df = pd.concat([df1, df2], ignore_index = True)
df

,English,Hindi,Hinglish,Average rating,Disagreement
0,Late Puranchandra Gupta never allowed any powe...,स्वर्गीय पूरनचन्द्र गुप्ता कभी भी अपने समाचारप...,late puranchandra gupta kabhi bhi apne press s...,7,4
1,From popular home stays where one can get a fi...,यहां के लोकप्रिय घरेलू आश्रय जहां आपको ग्रामीण...,yahan ke popular home aashray jahan aapko gram...,8,1
2,Compete with one another to achieve forgivenes...,तुम अपने परवरदिगार के (सबब) बख़्शिस की और बेहि...,tum apne paravardigar ke sabab bashis ki aur b...,3,5
3,Legislators must always remember that as repre...,विधायकों को सदैव याद रखना चाहिए कि जन प्रतिनिध...,legislators ko sadaiv yad people chahie ki pub...,8,1
4,The cap is designed to create an almost air ti...,कैप को इस तरह बनाया गया है कि यह गर्भाशय के मु...,cap ko is tarah banaya gaya hai ki yah garbhas...,9,2
...,...,...,...,...,...
1181,that God may distinguish the corrupt from the ...,ताकि अल्लाह नापाक को पाक से छाँटकर अलग करे और ...,taki allah napak ko pak se chhantakar alag kar...,4,7
1182,No such interface ‘% s'\n,कोई अंतरफलक ‘% s 'नहीं\n,koee interface s nhin\n,8,0
1183,Saccus endolymphaticus is also known as endoly...,अन्तरुदक कोश को अन्तः कर्णोदक कोश के रूप में भ...,antrudak sac ko anta endolymphatic sac ke roop...,7,1
1184,Retrieving quota information for folder '% s'\n,% s फ़ोल्डर के लिए के लिए कोटा प्राप्त कर रहा ...,s folder ke lie ke lie quota information kar r...,4,5


In [4]:
df = df[(df['Average rating'] >= 5) & (df['Disagreement'] <= 2)]
df = df.sample(n = 500, random_state = 1234)
print(len(df))

hindi_true = list(df['Hindi'])

500


In [5]:
hinglish_sentences = df['Hinglish'].values
print(type(hinglish_sentences))
print(hinglish_sentences.shape)

<class 'numpy.ndarray'>
(500,)


In [6]:
def calculate_scores(english_sentence, translation):
    reference = [english_sentence.split()]
    transformed_tokens = translation.split()

    bleu_score = sentence_bleu(reference, transformed_tokens)
    gleu_score = sentence_gleu(reference, transformed_tokens)
    met_score = meteor_score(reference, transformed_tokens)

    return bleu_score, gleu_score, met_score

In [7]:
# obtain just the transliterated text from the hinglish sentences

from google.transliteration import transliterate_word
def transliterate_all_alphanumeric_parts(sentence, lang_code='hi'):
    transliterated_sentence = []
    words = sentence.split()

    for word in words:
        # Use regex to find all alphanumeric parts in the word
        parts = re.findall(r'[a-zA-Z0-9]+|[^a-zA-Z0-9]+', word)
        
        transliterated_word = ""
        
        for part in parts:
            if part.isalnum():  # Transliterate only alphanumeric parts
                try:
                    # Transliterate the alphanumeric part
                    suggestions = transliterate_word(part, lang_code=lang_code)
                    transliterated_part = suggestions[0] if suggestions else part
                except IndexError:
                    # If transliteration fails, keep the alphanumeric part as is
                    transliterated_part = part
            else:
                # Keep non-alphanumeric parts (like "-" or spaces) as is
                transliterated_part = part
            
            transliterated_word += transliterated_part
        
        transliterated_sentence.append(transliterated_word)

    return ' '.join(transliterated_sentence)

lang = 'hi'
transliterated_sentences = []
for i in range(hinglish_sentences.size):
    transformed_sentence = transliterate_all_alphanumeric_parts(hinglish_sentences[i], lang_code=lang)
    transliterated_sentences.append(transformed_sentence)
    
    if i % 10 == 0:
        print("Transliterating sentence ", i+1)
        print('Original Code mixed text :', hinglish_sentences[i])
        print('Original Native Hindi convertion :', hindi_true[i])
        print('Transliterated to Hindi :', transformed_sentence)
        print("Sentences transliterated till now = ", len(transliterated_sentences))
        print()

Transliterating sentence  1
Original Code mixed text : un files not stored deta hai jo locally roop se jama nahin kie ge

Original Native Hindi convertion : उन फाइलों को लिखने देता है जो स्थानीय रूप से जमा नहीं किए गए

Transliterated to Hindi : उन फाइल्स नॉट स्टोर्ड देता है जो लोकालय रूप से जमा नहीं किए गए
Sentences transliterated till now =  1

Transliterating sentence  11
Original Code mixed text : aur yadi tum jhuthlate ho to tumse pahle many hi samuday jhuthla chuke nations noble par to bas keval spasht roop se  saty message  pahuncha dene ki responsible nations

Original Native Hindi convertion : और यदि तुम झुठलाते हो तो तुमसे पहले कितने ही समुदाय झुठला चुके है। रसूल पर तो बस केवल स्पष्ट रूप से (सत्य संदेश) पहुँचा देने की ज़िम्मेदारी है। "

Transliterated to Hindi : और यदि तुम झुठलाते हो तो तुमसे पहले मान्य ही समुदाय झुठला चुके नेशंस नोबल पर तो बस केवल स्पष्ट रूप से सत्य मैसेज पहुंचा देने की रेस्पोंसिबल नेशंस
Sentences transliterated till now =  11

Transliterating sentence  21
Or

In [8]:
transliterated_sentences

['उन फाइल्स नॉट स्टोर्ड देता है जो लोकालय रूप से जमा नहीं किए गए',
 'से हमारी स्पीकिंग २० स्टार्टेड सदी में शुरू हुई,',
 'यदि आप यह मानते हैं की आप कर सकते हैं , तो संभवता आप कर सकते हैं.',
 'प्रदेश ॉकपीस संभवतया उप महाद्वीप का संभतया सबसे ओल्डेस्ट पार्ट है जिसे गोंडवाना कहते हैं अर्थात गोंड समुदाय का घर.',
 'फिर जो कुछ वे ीर्ण रहे, वह उनके कुछ काम न आ सका',
 'फिर मैंने उनसे खुले तौर पर भी सीक्रेट की और उनसे चुपके-चुपके भी सीक्रेट की',
 'सोफ्री िसबन गियान्नी गाँधी और इंडिया सेण्टर में एक सेंचुरी १९९५ आईएसबीएन १-९००६२४-१२-५',
 'पोएट को लार्ड का ट्रुथ और ब्यूटी आस्पेक्ट अधिक प्रेरक फीलस है.',
 'असलियत में ये हुआ की गतियां काम हो गयीं',
 'सामान रूप से एविडेंट इंट्रोस्पेक्शन के लिए ग्राउंड्स हैं.',
 'और यदि तुम झुठलाते हो तो तुमसे पहले मान्य ही समुदाय झुठला चुके नेशंस नोबल पर तो बस केवल स्पष्ट रूप से सत्य मैसेज पहुंचा देने की रेस्पोंसिबल नेशंस',
 'इस काम के लिए हम इन्क्लूडिंग बेबीज कुछ पेशेंट्स के टेस्टिंग थे ब्लड कर रहे हैं.',
 'जनरल लिए ऋणदाता को वस्तुओं को बेचने का अधिकार प्रदान नहीं कर

In [9]:
import torch
import numpy as np
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)
from IndicTransToolkit import IndicProcessor

# Initialize metrics
indict2_bleu_scores = []
indict2_gleu_scores = []
indict2_meteor_scores = []

# Model and tokenizer setup
model_name = "ai4bharat/indictrans2-indic-en-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, trust_remote_code=True).to("cuda" if torch.cuda.is_available() else "cpu")
ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "hin_Deva", "eng_Latn"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

english_sentences = df['English'].values.tolist()

for i, hindi_text in enumerate(transliterated_sentences):
    try:
        # print(f"Processing sentence {i+1}...")
        
        # Preprocessing
        batch = ip.preprocess_batch(
            [hindi_text],
            src_lang=src_lang,
            tgt_lang=tgt_lang,
        )
        # print(f"Preprocessed batch: {batch}")
        
        # Tokenize the sentences and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=1,
                num_return_sequences=1,
            )
        # print(f"Generated tokens: {generated_tokens}")

        # Decode the generated tokens into text
        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )
        translation = ip.postprocess_batch(generated_tokens, lang=tgt_lang)[0]
        # print(translation)
        # print(english_sentences[i])
        # print()

        reference = [english_sentences[i].split()]
        transformed_tokens = translation.split()

        bleu_score = sentence_bleu(reference, transformed_tokens)
        gleu_score = sentence_gleu(reference, transformed_tokens)
        met_score = meteor_score(reference, transformed_tokens)
        
        indict2_bleu_scores.append(bleu_score)
        indict2_gleu_scores.append(gleu_score)
        indict2_meteor_scores.append(met_score)

        # Log progress every 10 sentences
        if i % 10 == 0:
            print("Translating sentence", i + 1)
            print("Original Hindi text:", hindi_text)
            print("Original English Conversion:", english_sentences[i])
            print("Translated to English:", translation)
            print("Average BLEU score:", np.mean(indict2_bleu_scores))
            print("Average GLEU score:", np.mean(indict2_gleu_scores))
            print("Average METEOR score:", np.mean(indict2_meteor_scores))
            print()

    except Exception as e:
        print(f"Error processing sentence {i+1}: {e}")
        continue


C:\Users\Nithish\anaconda3\envs\mt_final_project\Lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
C:\Users\Nithish\anaconda3\envs\mt_final_project\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Nithish\anaconda3\envs\mt_final_project\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, in

Translating sentence 1
Original Hindi text: उन फाइल्स नॉट स्टोर्ड देता है जो लोकालय रूप से जमा नहीं किए गए
Original English Conversion: Allows to burn files not stored locally

Translated to English: Not Stored gives you files that aren't locally submitted.
Average BLEU score: 1.2508498911928379e-231
Average GLEU score: 0.06666666666666667
Average METEOR score: 0.4383680555555556

Translating sentence 11
Original Hindi text: और यदि तुम झुठलाते हो तो तुमसे पहले मान्य ही समुदाय झुठला चुके नेशंस नोबल पर तो बस केवल स्पष्ट रूप से सत्य मैसेज पहुंचा देने की रेस्पोंसिबल नेशंस
Original English Conversion: “And if you deny, then many nations have denied before you; and the Noble Messenger is not responsible except to plainly convey (the message). ”

Translated to English: And if you lie, then the only way to clearly convey the truth message to the nations that have already been recognized as false before you is through Responsible Nations.
Average BLEU score: 0.1407330295262772
Average GLEU scor

In [10]:
average_bleu_score = np.mean(indict2_bleu_scores)
average_gleu_score = np.mean(indict2_gleu_scores)
average_meteor_score = np.mean(indict2_meteor_scores)
print('Average BLEU Score using IndicTrans2 model: ', average_bleu_score)
print('Average GLEU Score using IndicTrans2 model: ', average_gleu_score)
print('Average METEOR Score using IndicTrans2 model: ', average_meteor_score)

Average BLEU Score using IndicTrans2 model:  0.16769344441694095
Average GLEU Score using IndicTrans2 model:  0.2648356462448104
Average METEOR Score using IndicTrans2 model:  0.4886217530909094
